In [1]:
import sys

sys.path.insert(0, "..")

In [2]:
import numpy as np
import geopandas as gpd
import rasterio
from rasterio import plot

import matplotlib.pyplot as plt
import textwrap

from tests.raster_processing.clip_test import raster_path as SMALL_RASTER_PATH
from eis_toolkit.training_data_tools.points_to_raster import points_to_raster
from eis_toolkit.training_data_tools.random_sampling import generate_negatives

PATH_LABELS_GPKG = "../tests/data/remote/interpolating/interpolation_test_data_small.gpkg"
RASTER_PATH = "../tests/data/remote/small_raster.tif"

In [ ]:
gdf = gpd.read_file(PATH_LABELS_GPKG)
print(gdf)

In [3]:
def plot_samples(template_raster_path):
    
    with rasterio.open(template_raster_path) as temp_raster:
        
        outarray,outmeta = points_to_raster(positives=gdf,
                                            template_raster=temp_raster,
                                            nodata_value=-999)
        
        sampled_negatives, outarray2, _ = generate_negatives(raster_array=outarray,
                                                 raster_meta=outmeta,
                                                 sample_number=10,
                                                 random_seed=30)
        
        extent = [temp_raster.bounds.left, temp_raster.bounds.right,
                  temp_raster.bounds.bottom, temp_raster.bounds.top]

        fig,axes = plt.subplots(2,2,figsize=(20,15))
        norm = plt.Normalize(vmax=np.nanmax(outarray), vmin=np.nanmin(outarray))
        norm2 = plt.Normalize(vmax=np.nanmax(outarray2), vmin=np.nanmin(outarray2))

        plot.show(temp_raster, ax=axes[0][0])
        axes[0][0].set_title('Template Raster with positives', fontsize=15)
        points = gdf.cx[
                temp_raster.bounds.left : temp_raster.bounds.right,  
                temp_raster.bounds.bottom : temp_raster.bounds.top,  
            ]
        points.plot(ax=axes[0][0], facecolor='w', edgecolor='w')

        im1 = axes[0][1].imshow(outarray, cmap='gray', norm=norm, extent=extent)
        axes[0][1].set_title(textwrap.fill('True positives converted to raster.', width=30), fontsize=15)
        fig.colorbar(im1, ax=axes[0][1], orientation='vertical')

        plot.show(temp_raster, ax=axes[1][0])
        axes[1][0].set_title('Template Raster with positives and random negatives', fontsize=15)
        sampled_negatives.plot(ax=axes[1][0], facecolor='r', edgecolor='r')
        points.plot(ax=axes[1][0], facecolor='w', edgecolor='w')

        im1 = axes[1][1].imshow(outarray2, cmap='gray', norm=norm2, extent=extent)
        axes[1][1].set_title(textwrap.fill('True positives and sampled negatives converted to raster.', width=30), fontsize=15)
        fig.colorbar(im1, ax=axes[1][1], orientation='vertical')

        plt.tight_layout()
        plt.show()

In [4]:
plot_samples(SMALL_RASTER_PATH)

NameError: name 'gdf' is not defined